# 🔬 Advanced HyperCat Examples

This notebook demonstrates various applications and use cases of the HyperCat category theory library, showcasing its capabilities through concrete mathematical examples.

In [ ]:
from hypercat import Category, Object, Morphism, Functor, NaturalTransformation
from hypercat.categories import StandardCategories
from hypercat.higher import TwoCell, TwoCategory
import json

## Example 1: Monoid as Category

In [ ]:
def example_1_monoid_as_category():
    """Representing a monoid as a single-object category."""
    print("=== Example 1: Monoid as Category ===")
    
    # Create a category representing the monoid (ℤ/3ℤ, +)
    monoid_cat = Category("Z3_Monoid")
    
    # Single object representing the monoid
    star = Object("*")
    monoid_cat.add_object(star)
    
    # Morphisms represent monoid elements
    m0 = Morphism("0", star, star)  # identity element
    m1 = Morphism("1", star, star)
    m2 = Morphism("2", star, star)
    
    monoid_cat.add_morphism(m0).add_morphism(m1).add_morphism(m2)
    
    # Set up composition (addition mod 3)
    monoid_cat.set_composition(m0, m0, m0)  # 0 + 0 = 0
    monoid_cat.set_composition(m0, m1, m1)  # 1 + 0 = 1
    monoid_cat.set_composition(m1, m0, m1)  # 0 + 1 = 1
    monoid_cat.set_composition(m0, m2, m2)  # 2 + 0 = 2
    monoid_cat.set_composition(m2, m0, m2)  # 0 + 2 = 2
    
    # Non-trivial compositions
    monoid_cat.set_composition(m1, m1, m2)  # 1 + 1 = 2
    monoid_cat.set_composition(m1, m2, m0)  # 2 + 1 = 0
    monoid_cat.set_composition(m2, m1, m0)  # 1 + 2 = 0
    monoid_cat.set_composition(m2, m2, m1)  # 2 + 2 = 1
    
    print(f"Monoid category: {len(monoid_cat.objects)} object, {len(monoid_cat.morphisms)} morphisms")
    
    # Show composition table
    print("\nComposition table (g ∘ f):")
    morphs = [m0, m1, m2]
    for g in morphs:
        for f in morphs:
            comp = monoid_cat.compose(f, g)
            if comp:
                print(f"  {g.name} ∘ {f.name} = {comp.name}")
    
    return monoid_cat

monoid = example_1_monoid_as_category()

## Example 2: Category of Directed Graphs

In [ ]:
def example_2_graph_category():
    """Category of directed graphs and graph homomorphisms."""
    print("=== Example 2: Category of Directed Graphs ===")
    
    graph_cat = Category("Graph")
    
    # Create some graph objects
    empty_graph = Object("Empty", {"vertices": set(), "edges": set()})
    single_vertex = Object("Single", {"vertices": {1}, "edges": set()})
    loop_graph = Object("Loop", {"vertices": {1}, "edges": {(1,1)}})
    two_vertex_graph = Object("TwoVertex", {"vertices": {1,2}, "edges": {(1,2)}})
    
    graph_cat.add_object(empty_graph)
    graph_cat.add_object(single_vertex)
    graph_cat.add_object(loop_graph)
    graph_cat.add_object(two_vertex_graph)
    
    # Graph homomorphisms (vertex mappings that preserve edges)
    empty_to_single = Morphism("empty→single", empty_graph, single_vertex)
    single_to_loop = Morphism("single→loop", single_vertex, loop_graph)
    loop_to_two = Morphism("loop→two", loop_graph, two_vertex_graph, 
                          data={"vertex_map": {1: 1}})
    
    graph_cat.add_morphism(empty_to_single)
    graph_cat.add_morphism(single_to_loop)
    graph_cat.add_morphism(loop_to_two)
    
    print(f"Graph category: {len(graph_cat.objects)} objects, {len(graph_cat.morphisms)} morphisms")
    
    # Show the graphs
    for obj in graph_cat.objects:
        if obj.data:
            print(f"  {obj.name}: V={obj.data.get('vertices', set())}, E={obj.data.get('edges', set())}")
    
    return graph_cat

graph_cat = example_2_graph_category()

## Example 3: Functor FinSet → Vect

In [ ]:
def example_3_functor_between_categories():
    """Functor from finite sets to vector spaces."""
    print("=== Example 3: Functor FinSet → Vect ===")
    
    # Source category: FinSet (simplified)
    finset = Category("FinSet")
    
    set1 = Object("Set1", {"elements": {1}})
    set2 = Object("Set2", {"elements": {1, 2}})
    set3 = Object("Set3", {"elements": {1, 2, 3}})
    
    finset.add_object(set1).add_object(set2).add_object(set3)
    
    # Functions between sets
    f12 = Morphism("f12", set1, set2, data={"function": {1: 1}})
    f23 = Morphism("f23", set2, set3, data={"function": {1: 1, 2: 3}})
    f13 = Morphism("f13", set1, set3, data={"function": {1: 1}})
    
    finset.add_morphism(f12).add_morphism(f23).add_morphism(f13)
    finset.set_composition(f12, f23, f13)  # f23 ∘ f12 = f13
    
    # Target category: Vect (simplified)
    vect = Category("Vect")
    
    k1 = Object("k¹", {"dimension": 1})  # 1-dimensional vector space
    k2 = Object("k²", {"dimension": 2})  # 2-dimensional vector space
    k3 = Object("k³", {"dimension": 3})  # 3-dimensional vector space
    
    vect.add_object(k1).add_object(k2).add_object(k3)
    
    # Linear maps
    lin12 = Morphism("L12", k1, k2, data={"matrix": [[1], [0]]})
    lin23 = Morphism("L23", k2, k3, data={"matrix": [[1, 0], [0, 0], [0, 1]]})
    lin13 = Morphism("L13", k1, k3, data={"matrix": [[1], [0], [0]]})
    
    vect.add_morphism(lin12).add_morphism(lin23).add_morphism(lin13)
    vect.set_composition(lin12, lin23, lin13)
    
    # Create the free vector space functor
    free_functor = Functor("Free", finset, vect)
    
    # Map objects: n-element set → n-dimensional vector space
    free_functor.map_object(set1, k1)
    free_functor.map_object(set2, k2)
    free_functor.map_object(set3, k3)
    
    # Map morphisms: functions → linear maps
    free_functor.map_morphism(f12, lin12)
    free_functor.map_morphism(f23, lin23)
    free_functor.map_morphism(f13, lin13)
    
    print(f"Free functor from {finset.name} to {vect.name}")
    print(f"Maps {len(finset.objects)} objects to {len(vect.objects)} objects")
    print(f"Valid functor: {free_functor.is_functor()}")
    
    return free_functor

free_functor = example_3_functor_between_categories()

## Example 4: Natural Transformation

In [ ]:
def example_4_natural_transformation():
    """Natural transformation between functors."""
    print("=== Example 4: Natural Transformation ===")
    
    # Create category C with two objects
    C = Category("C")
    A = Object("A")
    B = Object("B")
    C.add_object(A).add_object(B)
    
    f = Morphism("f", A, B)
    C.add_morphism(f)
    
    # Create category D (same structure)
    D = Category("D")
    X = Object("X")
    Y = Object("Y")
    D.add_object(X).add_object(Y)
    
    g = Morphism("g", X, Y)
    h = Morphism("h", X, Y)  # Another morphism X → Y
    D.add_morphism(g).add_morphism(h)
    
    # Create two functors F, G: C → D
    F = Functor("F", C, D)
    F.map_object(A, X).map_object(B, Y)
    F.map_morphism(f, g)
    
    G = Functor("G", C, D)
    G.map_object(A, X).map_object(B, Y)
    G.map_morphism(f, h)
    
    # Create natural transformation α: F ⇒ G using identity components
    alpha = NaturalTransformation("α", F, G)
    
    # Use existing identity morphisms
    alpha.set_component(A, D.identities[X])  # Component at A (identity on X)
    alpha.set_component(B, D.identities[Y])  # Component at B (identity on Y)
    
    print(f"Natural transformation {alpha.name}: {F.name} ⇒ {G.name}")
    print(f"Components defined at {len(alpha.components)} objects")
    
    return alpha

nat_trans = example_4_natural_transformation()

## Example 5: Walking Structures

In [ ]:
def example_5_walking_structures():
    """Various walking structures and their universal properties."""
    print("=== Example 5: Walking Structures ===")
    
    # Walking arrow (standard arrow category)
    arrow_cat = StandardCategories.arrow_category()
    print(f"Walking arrow: {len(arrow_cat.objects)} objects, {len(arrow_cat.morphisms)} morphisms")
    
    # Walking commutative square
    comm_square = Category("WalkingCommSquare")
    
    # Objects: corners of the square
    top_left = Object("TL")
    top_right = Object("TR")
    bottom_left = Object("BL")
    bottom_right = Object("BR")
    
    comm_square.add_object(top_left).add_object(top_right)
    comm_square.add_object(bottom_left).add_object(bottom_right)
    
    # Morphisms: edges of the square
    top = Morphism("top", top_left, top_right)
    bottom = Morphism("bottom", bottom_left, bottom_right)
    left = Morphism("left", top_left, bottom_left)
    right = Morphism("right", top_right, bottom_right)
    
    # Diagonal composition (both paths equal)
    diag = Morphism("diag", top_left, bottom_right)
    
    comm_square.add_morphism(top).add_morphism(bottom)
    comm_square.add_morphism(left).add_morphism(right)
    comm_square.add_morphism(diag)
    
    # Set up commutativity: both paths give the same result
    comm_square.set_composition(top, right, diag)    # right ∘ top
    comm_square.set_composition(left, bottom, diag)  # bottom ∘ left
    
    print(f"Walking commutative square: {len(comm_square.objects)} objects, {len(comm_square.morphisms)} morphisms")
    
    return comm_square

walking = example_5_walking_structures()

## Example 6: 2-Category

In [ ]:
def example_6_two_category():
    """2-category with 2-cells (natural transformations)."""
    print("=== Example 6: 2-Category Example ===")
    
    # Create a simple 2-category
    two_cat = TwoCategory("Cat")
    
    # 0-cells (objects): small categories
    C = Object("C")
    D = Object("D")
    two_cat.add_object(C).add_object(D)
    
    # 1-cells (morphisms): functors
    F = Morphism("F", C, D)
    G = Morphism("G", C, D)
    two_cat.add_morphism(F).add_morphism(G)
    
    # 2-cells: natural transformations
    alpha = TwoCell("α", F, G)  # α: F ⇒ G
    beta = TwoCell("β", G, F)   # β: G ⇒ F
    
    two_cat.add_two_cell(alpha).add_two_cell(beta)
    
    # Identity 2-cells
    id_F = TwoCell("id_F", F, F)
    id_G = TwoCell("id_G", G, G)
    two_cat.add_two_cell(id_F).add_two_cell(id_G)
    
    print(f"2-category {two_cat.name}:")
    print(f"  0-cells (objects): {len(two_cat.objects)}")
    print(f"  1-cells (morphisms): {len(two_cat.morphisms)}")
    print(f"  2-cells: {len(two_cat.two_cells)}")
    
    for cell in two_cat.two_cells:
        print(f"    {cell}")
    
    return two_cat

two_cat = example_6_two_category()

## Example 7: Products and Coproducts

In [ ]:
def example_7_limits_and_colimits():
    """Computing limits and colimits (products and coproducts)."""
    print("=== Example 7: Products and Coproducts ===")
    
    # Create a category with products
    prod_cat = Category("Set")  # Simplified Set category
    
    A = Object("A", {"elements": {"a1", "a2"}})
    B = Object("B", {"elements": {"b1", "b2"}})
    AxB = Object("A×B", {"elements": {("a1","b1"), ("a1","b2"), ("a2","b1"), ("a2","b2")}})
    
    prod_cat.add_object(A).add_object(B).add_object(AxB)
    
    # Projection morphisms
    pi1 = Morphism("π₁", AxB, A, data={"function": "first projection"})
    pi2 = Morphism("π₂", AxB, B, data={"function": "second projection"})
    
    prod_cat.add_morphism(pi1).add_morphism(pi2)
    
    print(f"Product category example:")
    print(f"  A = {A.data['elements']}")
    print(f"  B = {B.data['elements']}")
    print(f"  A×B = {AxB.data['elements']}")
    print(f"  Projections: {pi1.name}, {pi2.name}")
    
    # Coproduct (disjoint union)
    A_plus_B = Object("A+B", {"elements": {("inl","a1"), ("inl","a2"), ("inr","b1"), ("inr","b2")}})
    prod_cat.add_object(A_plus_B)
    
    # Injection morphisms
    inl = Morphism("inl", A, A_plus_B, data={"function": "left injection"})
    inr = Morphism("inr", B, A_plus_B, data={"function": "right injection"})
    
    prod_cat.add_morphism(inl).add_morphism(inr)
    
    print(f"  A+B = {A_plus_B.data['elements']}")
    print(f"  Injections: {inl.name}, {inr.name}")
    
    return prod_cat

limits = example_7_limits_and_colimits()

## Example 8: Adjoint Functors

In [ ]:
def example_8_adjoint_functors():
    """Adjoint functors (Free-Forgetful adjunction)."""
    print("=== Example 8: Adjoint Functors ===")
    
    # Set category (simplified)
    Set = Category("Set")
    X = Object("X", {"elements": {1, 2}})
    Y = Object("Y", {"elements": {"a"}})
    Set.add_object(X).add_object(Y)
    
    f = Morphism("f", X, Y, data={"function": {1: "a", 2: "a"}})
    Set.add_morphism(f)
    
    # Group category (simplified)
    Grp = Category("Grp")
    FX = Object("F(X)", {"generators": {1, 2}, "relations": []})  # Free group on X
    Z = Object("Z", {"elements": [0, 1, -1], "operation": "addition"})  # Cyclic group
    Grp.add_object(FX).add_object(Z)
    
    h = Morphism("h", FX, Z, data={"homomorphism": {1: 1, 2: 1}})
    Grp.add_morphism(h)
    
    # Free functor F: Set → Grp
    Free = Functor("Free", Set, Grp)
    Free.map_object(X, FX)
    Free.map_object(Y, Object("F(Y)", {"generators": {"a"}, "relations": []}))
    
    # Forgetful functor U: Grp → Set
    Forget = Functor("Forget", Grp, Set)
    UZ = Object("U(Z)", {"elements": [0, 1, -1]})  # Underlying set of Z
    Set.add_object(UZ)
    
    Forget.map_object(Z, UZ)
    Forget.map_object(FX, Object("U(F(X))", {"elements": "words in {1,2}"}))
    
    print(f"Free-Forgetful adjunction:")
    print(f"  Free functor: {Set.name} → {Grp.name}")
    print(f"  Forgetful functor: {Grp.name} → {Set.name}")
    print(f"  F(X) = free group on {X.data['elements']}")
    print(f"  U(Z) = underlying set {UZ.data['elements']}")
    
    # The adjunction gives us: Hom_Grp(F(X), Z) ≅ Hom_Set(X, U(Z))
    print(f"  Adjunction: Hom_Grp(F(X), Z) ≅ Hom_Set(X, U(Z))")
    
    return Free, Forget

free, forget = example_8_adjoint_functors()

## Example 9: Elementary Topos

In [ ]:
def example_9_topos_structure():
    """Elementary topos structure (simplified)."""
    print("=== Example 9: Elementary Topos (Simplified) ===")
    
    # Create a simple topos-like category
    topos = Category("SimpleTopos")
    
    # Terminal object
    one = Object("1", {"elements": {"*"}})
    
    # Subobject classifier
    omega = Object("Ω", {"elements": {"true", "false"}})
    
    # Some other objects
    A = Object("A", {"elements": {"a1", "a2"}})
    B = Object("B", {"elements": {"b1"}})
    
    topos.add_object(one).add_object(omega).add_object(A).add_object(B)
    
    # Truth value morphism
    true_morph = Morphism("true", one, omega, 
                         data={"function": {"*": "true"}})
    
    # Characteristic functions (simplified)
    chi_A = Morphism("χ_A", A, omega, 
                     data={"function": {"a1": "true", "a2": "false"}})
    
    topos.add_morphism(true_morph).add_morphism(chi_A)
    
    print(f"Elementary topos structure:")
    print(f"  Terminal object: {one.name} = {one.data['elements']}")
    print(f"  Subobject classifier: {omega.name} = {omega.data['elements']}")
    print(f"  Truth morphism: {true_morph.name}")
    print(f"  Characteristic function: {chi_A.name}")
    print(f"  Has exponentials, pullbacks, and subobject classifier")
    
    return topos

topos = example_9_topos_structure()

## Summary

This notebook has demonstrated 9 advanced category theory concepts using HyperCat:

1. ✅ Monoid as single-object category
2. ✅ Category of directed graphs
3. ✅ Free vector space functor
4. ✅ Natural transformations
5. ✅ Walking structures (universal properties)
6. ✅ 2-categories with 2-cells
7. ✅ Products and coproducts
8. ✅ Adjoint functors (Free-Forgetful)
9. ✅ Elementary topos structure

The HyperCat library supports:
- Categories, functors, natural transformations
- Higher categories (2-categories)
- Standard constructions (limits, colimits)
- Universal properties and adjunctions
- Topos theory foundations